In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('Dataset\\Train\\train_data.csv')
X_train = data.iloc[:, :49].values
y_train = data.iloc[:, -1].values
X_train = X_train.reshape((X_train.shape[0], 7, 7, 1))

In [3]:
def create_cnn(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.AveragePooling2D((1, 1)))
    model.add(layers.BatchNormalization())
    return model

In [4]:
def create_lstm(input_shape):
    model = models.Sequential()
    model.add(layers.Reshape((49,1), input_shape=input_shape))
    model.add(layers.LSTM(32, activation='tanh'))
    model.add(layers.Reshape((1, 1, 32)))
    return model

In [5]:
def create_autoencoder(input_shape):
    model = models.Sequential()

    # Encoder
    model.add(layers.Dense(256, input_shape= input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    
    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    
    model.add(layers.Dense(64))
    model.add(layers.Dense(32))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    
    
    model.add(layers.Dense(32))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    
    model.add(layers.Dense(64))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.MaxPooling2D((2,2)))
    # Output layer
    model.add(layers.Reshape((1,1,64)))

    return model

In [6]:
def create_full_model(input_shape):
    autoencoder_model = create_autoencoder(input_shape)
    cnn_model = create_cnn(input_shape)
    cnn_model_1 = create_cnn(input_shape)
    cnn_model_2 = create_cnn(input_shape)
    lstm_model = create_lstm(input_shape)
    
    concatenated_outputs = layers.concatenate([autoencoder_model.output, cnn_model.output, cnn_model_1.output, cnn_model_2.output, lstm_model.output])
    flattened_outputs = layers.Flatten()(concatenated_outputs)
    reshaped_outputs = layers.Reshape((flattened_outputs.shape[1], 1))(flattened_outputs)
    flatten_layer = layers.Flatten(input_shape=(None, 288, 1))(reshaped_outputs)
    dense_layer_1 = layers.Dense(256, activation='relu')(flatten_layer)
    dense_layer_2 = layers.Dense(128, activation='relu')(dense_layer_1)
    dropout_layer = layers.Dropout(0.5)(dense_layer_2)
    dense_layer_3 = layers.Dense(64, activation='relu')(dropout_layer)
    dense_layer_4 = layers.Dense(32, activation='relu')(dense_layer_3)
    batch_norm_layer = layers.BatchNormalization()(dense_layer_4)
    output_layer = layers.Dense(18, activation='softmax')(batch_norm_layer)
    model_inputs = [autoencoder_model.input, cnn_model.input, cnn_model_1.input, cnn_model_2.input, lstm_model.input]
    model_outputs = output_layer

    # Create the full model
    full_model = models.Model(inputs=model_inputs, outputs=model_outputs)
    return full_model

In [9]:
def iterative_training(X_train,y_train,threshold=0.90):
    input_shape = (7,7,1)
    full_model = create_full_model(input_shape)
    custom_optimizer = Adam(learning_rate=0.001)
    full_model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    iteration = 0
    accuracy = 0.0
    
    X_train_iter, X_val, y_train_iter, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.1, random_state=42)
    while accuracy < threshold:
        # Train the model for the current iteration
        early_stopping_loss = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, mode='min')
        early_stopping_accuracy = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, mode='max')
        full_model.fit([X_train_iter, X_train_iter, X_train_iter, X_train_iter, X_train_iter], y_train_iter, epochs=25, batch_size=512, validation_data=([X_val, X_val, X_val, X_val, X_val], y_val), callbacks=[early_stopping_loss, early_stopping_accuracy], verbose=1)

        # Evaluate the model on the validation set
        _, accuracy = full_model.evaluate([X_train_iter, X_train_iter, X_train_iter, X_train_iter, X_train_iter], y_train_iter)
        rounded_accuracy = round(accuracy * 100, 2)
        print(f"Accuracy : {rounded_accuracy}%")
        # Save the trained sub-model
        full_model.save(os.path.join("Models\\SubModel", f'sub_model_{iteration}_{rounded_accuracy}.h5'))
        
        probabilities = full_model.predict([X_train_iter, X_train_iter, X_train_iter, X_train_iter, X_train_iter])
        predicted_classes = np.argmax(probabilities, axis=1)
        correct_indices = np.where(predicted_classes == y_train_iter)[0]
        incorrect_indices = np.where(predicted_classes != y_train_iter)[0]
        X_train_iter_misclassified = X_train_iter[incorrect_indices]
        y_train_iter_misclassified = y_train[incorrect_indices]
        X_train_iter, _, y_train_iter, _ = train_test_split(np.concatenate((X_train_iter[correct_indices], X_train_iter_misclassified )),np.concatenate((y_train_iter[correct_indices],y_train_iter_misclassified)), test_size=0.1, random_state=42)
        iteration += 1

    print("Training completed.")

In [10]:
iterative_training(X_train, y_train)

Epoch 1/25
238/238 [==============================] - 59s 214ms/step - loss: 1.7388 - accuracy: 0.4841 - val_loss: 1.4534 - val_accuracy: 0.5716
Epoch 2/25
238/238 [==============================] - 44s 186ms/step - loss: 0.9694 - accuracy: 0.6770 - val_loss: 0.9373 - val_accuracy: 0.6626
Epoch 3/25
238/238 [==============================] - 42s 177ms/step - loss: 0.7668 - accuracy: 0.7170 - val_loss: 0.7134 - val_accuracy: 0.7341
Epoch 4/25
238/238 [==============================] - 41s 173ms/step - loss: 0.7271 - accuracy: 0.7239 - val_loss: 0.7514 - val_accuracy: 0.7122
Epoch 5/25
238/238 [==============================] - 41s 171ms/step - loss: 0.6944 - accuracy: 0.7328 - val_loss: 0.8105 - val_accuracy: 0.6775
Epoch 6/25
238/238 [==============================] - 41s 174ms/step - loss: 0.6656 - accuracy: 0.7398 - val_loss: 1.0662 - val_accuracy: 0.6133
Epoch 7/25
238/238 [==============================] - 44s 183ms/step - loss: 0.6601 - accuracy: 0.7416 - val_loss: 0.8451 - val_ac